<a href="https://colab.research.google.com/github/Pistolll/praktika2_TMOC/blob/main/1prak_custom_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ихтиёр М. ББМО-01-24

In [1]:
# (numpy предоставляет поддержку многомерных массивов и математических операций)
import numpy as np

# (pandas используется для работы с табличными данными, например, DataFrame)
import pandas as pd

In [ ]:
# use mushroom dataset\\\\используем датасет

In [4]:
url = "https://raw.githubusercontent.com/Pistolll/praktika2_TMOC/refs/heads/main/%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0%202/01-custom-naive-bayes/mushrooms.csv"
df = pd.read_csv(url)

In [5]:
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [6]:
df.shape

(8124, 23)

In [7]:
# convert in numeric data
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [11]:
le = LabelEncoder() # Создаём экземпляр LabelEncoder для кодирования меток //import lableencoder

In [12]:
# Применяем LabelEncoder ко всем столбцам DataFrame,
# преобразуя текстовые значения в числовые
ds = df.apply(func=le.fit_transform)

In [13]:
ds.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [14]:
# Преобразуем DataFrame `ds` в массив NumPy (матрицу чисел)
# Теперь `data` — это двумерный массив, где строки = наблюдения, столбцы = признаки
data = ds.values

# Формируем матрицу признаков `X`:
# - Берем все строки (`:`) и все столбцы, кроме первого (`1:`)
X = data[:, 1:]

# Формируем вектор целевой переменной `y`:
# - Берем все строки (`:`) и только первый столбец (`0`)
# - Это метки классов для обучения модели
y = data[:, 0]

In [16]:
X.shape, y.shape

((8124, 22), (8124,))

In [17]:
# - X - матрица признаков
# - y - вектор целевых значений
# - test_size=0.33 - 33% данных пойдут в тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Custom NB

In [18]:
# Кастомная реализация наивного байесовского классификатора
class CustomNB:
    def fit(self, X, y):
        # Сохраняем обучающие данные
        self.X_train = X  # Матрица признаков
        self.y_train = y  # Вектор меток классов

    # Вычисление априорной вероятности P(y=label)
    def prior_prob(self, label):
        total = self.y_train.shape[0]  # Общее количество примеров
        class_examples = np.sum(self.y_train == label)  # Количество примеров данного класса
        return class_examples / float(total)  # Вероятность класса

    # Вычисление условной вероятности P(Xi=val|y=label)
    def conditional_prob(self, feature_col, feature_val, label):
        # Фильтруем примеры по классу
        X_filtered = self.X_train[self.y_train==label]  # Примеры только нужного класса
        # Считаем сколько примеров имеют нужное значение признака
        numerator = np.sum(X_filtered[:, feature_col] == feature_val)
        denominator = len(X_filtered)  # Общее количество примеров класса
        return numerator / denominator  # Вероятность признака при данном классе

    # Предсказание для одного примера
    def predict_point(self, X_test):
        classes = np.unique(self.y_train)  # Уникальные классы
        n_features = self.X_train.shape[1]  # Количество признаков
        post_pro = []  # Для хранения апостериорных вероятностей

        # Вычисляем апостериорную вероятность для каждого класса
        for label in classes:
            likehood = 1.0  # Инициализируем likelihood
            # Вычисляем произведение условных вероятностей для всех признаков
            for feature in range(n_features):
                cond = self.conditional_prob(feature, X_test[feature], label)
                likehood *= cond
            prior = self.prior_prob(label)  # Априорная вероятность
            post = prior * likehood  # Апостериорная вероятность
            post_pro.append(post)

        # Возвращаем класс с максимальной вероятностью
        return np.argmax(post_pro)

    # Предсказание для множества примеров
    def predict(self, X_test):
        result = []
        for point in X_test:  # Для каждого примера в тестовой выборке
            result.append(self.predict_point(point))
        return np.array(result)  # Возвращаем массив предсказаний

    # Оценка точности модели
    def score(self, X_test, y_test):
        # Сравниваем предсказания с истинными метками и вычисляем среднюю точность
        return (self.predict(X_test) == y_test).mean()

In [25]:
# Создаем экземпляр нашего кастомного наивного байесовского классификатора
model = CustomNB()  # Инициализация модели, но без обучения

In [26]:
# Обучаем модель на тренировочных данных
model.fit(X_train, y_train)

In [27]:
# Делаем предсказания для первых 10 тестовых примеров
model.predict(X_test[:10])

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [28]:
# Получаем первые 10 истинных значений целевой переменной из тестового набора
y_test[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [29]:
# Оцениваем точность модели на тестовых данных
model.score(X_test, y_test)

np.float64(0.9973890339425587)